In [6]:
%load_ext autoreload

In [7]:
%autoreload 2

In [4]:
import pyspark as psk
from pyspark.sql import SparkSession
import pandas as pd


In [8]:
from nomad.daphmeIO import _is_traj_df_spark

In [9]:
spark = SparkSession.builder \
    .master("local[12]") \
    .appName("Tests") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", 
            "com.amazonaws.auth.DefaultAWSCredentialsProviderChain") \
    .config("spark.jars.packages", 
            "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .getOrCreate()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [10]:
traj_cols = {'user_id': 'uid',
             'latitude': 'latitude',
             'longitude': 'longitude',
             'timestamp': 'timestamp'}

In [11]:
df = spark.read.parquet("s3a://catalog-csslab/garden-city-45k/device-level/admin_1_id=GC/admin_2_id=GC.CD/event_start_date_utc=2024-01-13/")
df.show(10)

+--------------------+----------+------------------+-------------------+---------------+
|                 uid| timestamp|          latitude|          longitude|timezone_offset|
+--------------------+----------+------------------+-------------------+---------------+
|dazzling_friendly...|1705165320|  38.3217495027236| -36.66645623726486|          -3600|
|dazzling_friendly...|1705165380| 38.32171358576374|-36.666469104355116|          -3600|
|dazzling_friendly...|1705166640|38.321709902716684|-36.666493842217406|          -3600|
|dazzling_friendly...|1705166700| 38.32166160047268| -36.66641416123398|          -3600|
|dazzling_friendly...|1705167420| 38.32161312650487|-36.666508981783856|          -3600|
|dazzling_friendly...|1705167660| 38.32182592672064|  -36.6663535995203|          -3600|
|dazzling_friendly...|1705167780| 38.32171345553894| -36.66645434557337|          -3600|
|dazzling_friendly...|1705168020|38.321888231640166| -36.66651289047773|          -3600|
|dazzling_friendly...

In [27]:
first_timestamp = df.select('timestamp').take(1)
len(str(first_timestamp[0]['timestamp']))

10

In [19]:
_is_traj_df_spark(df)

True

In [15]:
pd_df = pd.read_csv('./daphme/nomad/data/gc_sample.csv')

In [28]:
(isinstance(pd_df, psk.sql.dataframe.DataFrame))

False

In [19]:
# Create a spark dataframe from pandas
spark.createDataFrame(pd_df).show()

+---------------+----------+-----------------+-------------------+
|            uid| timestamp|         latitude|          longitude|
+---------------+----------+-----------------+-------------------+
|wizardly_joliot|1704119340| 38.3217114212002|  -36.6673339303945|
|wizardly_joliot|1704119700|38.32167577212169| -36.66736487400436|
|wizardly_joliot|1704155880|38.32095880607729|-36.666748241131394|
|wizardly_joliot|1704156000|38.32093630091352| -36.66673870270966|
|wizardly_joliot|1704156840|38.32092370280456|  -36.6667470974188|
|wizardly_joliot|1704157860|38.32099369377554|  -36.6667731493393|
|wizardly_joliot|1704157920|38.32096445136562| -36.66676255801932|
|wizardly_joliot|1704158820|38.32092139039607| -36.66674990860071|
|wizardly_joliot|1704161280|38.32097084909334| -36.66680909863967|
|wizardly_joliot|1704183120|38.32175777610523| -36.66643421364937|
|wizardly_joliot|1704184320|38.32162577076888| -36.66641575510961|
|wizardly_joliot|1704185400|38.32168492217367| -36.66645400646

In [28]:
spark.stop()